In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

2023-04-21 16:12:29.587585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 16:12:29.707195: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
# SwiLU 함수 정의 (alpha값 항상 1로 고정)
def serlu(x, lamda = 1.0786, alpha= 1):
    return tf.where(x>=0, x, alpha * x * tf.exp(x))

In [4]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

2023-04-21 16:12:31.894912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 16:12:31.897530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 16:12:31.897816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 16:12:31.898411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
# 모델 컴파일
opt = RMSprop(learning_rate=1e-4)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [10]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR10", entity="hcim", name='LeNet-swilu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))

callbacks = [wandbCallback]

In [11]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Epoch 1/200


2023-04-21 16:14:56.274737: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model/conv2d_3/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-21 16:14:56.645048: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


386/391 [============================>.] - ETA: 0s - loss: 2.0169 - accuracy: 0.2612

2023-04-21 16:15:02.657941: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model_1/model/conv2d_3/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best/assets


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best)... Done. 0.0s


391/391 [==============================] - 12s 31ms/step - loss: 0.5145 - accuracy: 0.8182 - val_loss: 0.7229 - val_accuracy: 0.7539
Epoch 200/200
390/391 [============================>.] - ETA: 0s - loss: 0.5127 - accuracy: 0.8184

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_161448-tzux4skn/files/model-best)... Done. 0.0s


391/391 [==============================] - 12s 30ms/step - loss: 0.5125 - accuracy: 0.8184 - val_loss: 0.7212 - val_accuracy: 0.7547
